In [ ]:
#check for customer columns not in general pop column
customer_col = [i for i in customer_data.columns.to_list() if i not in train_data.columns.to_list()]
to_drop = ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']
train_data.drop(columns=to_drop, inplace=True)


#create a dataframe showing number of missing values for each feature
null_columns = (np.count_nonzero(train_data[train_data.columns].isnull(), axis = 0))
null_columns_df = pd.DataFrame(null_columns,index=train_data.columns)
null_columns_df = null_columns_df.sort_values(0,ascending=False)
null_columns_df = null_columns_df.rename(columns={0:'no of missing values'})

#poltting bar graph for features with 17 most missing values
import matplotlib.pyplot as plt   
Features = null_columns_df.index[0:17].to_list()[::-1]
Quantity = null_columns_df['no of missing values'][0:17].to_list()[::-1]

plt.barh(Features,Quantity)
plt.title('FEATURES WITH MISSING VALUES')
plt.ylabel('FEATURES')
plt.xlabel('NO OF MISSING VALUES')
plt.show()

#create a dictionary with features as key and number of missing values as value
column_names = train_data.columns
null_info =dict()
for i, j in zip(null_columns, column_names):
    if i > 0:
        null_info[j] = i

null_info_sorted = dict(sorted(null_info.items(), key=lambda item: item[1], reverse=True))

#drop columns with 400000 or more missing values
columns_to_drop = [i for i in null_info if null_info[i] >= 400000]
train_data.drop(columns = columns_to_drop, inplace=True)


#replace unknown values (-1, 0, 9) with nan by searching for unknown values in diss_attribute
var_unknown = {}
dias_att['Value'][0]
for i,j,k in zip(dias_att['Attribute'], dias_att['Meaning'], dias_att['Value']):
    if i != np.nan and j == 'unknown':
        if type(k) == str:
            val = [int(i) for i in k.split(",")]

            var_unknown[i] = val
        else:
            var_unknown[i] = k

#search for features that are in dias attribute but not in azdias dataset
var_unknown_col = [i for i in var_unknown if i not in train_data.columns.to_list()]

#rename columns
columns_to_rename = ['CAMEO_DEUINTL_2015', 'KBA13_CCM_1400_2500', 'SOHO_FLAG', np.nan]
columns_renamed = ['CAMEO_INTL_2015', 'KBA13_CCM_1401_2500', 'SOHO_KZ']
columns_to_remove = ['BIP_FLAG', 'D19_KK_KUNDENTYP', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'WACHSTUMSGEBIET_NB']

k = 0
for i,j in var_unknown.items():
    if i in columns_to_remove:
        continue
    if i in columns_to_rename:
        train_data[columns_renamed[k]] = train_data[columns_renamed[k]].replace(j, np.nan)
        continue
    train_data[i] = train_data[i].replace(j, np.nan)

#check if -1 values have been replaced
(np.count_nonzero(train_data[train_data.columns]==-1, axis = 0))

#for value 9, there are features with value 9 that is unknown. Therefore i manually check each feature that have value 9.
val_9 = (np.count_nonzero(train_data[train_data.columns]==9, axis = 0))
column_names = train_data.columns
val_9_1 ={}
for i, j in zip(val_9, column_names):
    if i > 0:
        val_9_1[j] = i
#after checking, all features with unknown values -1 and 9 have been converted into NaN.

#'RELAT_AB' need to replace 9 with nan since the azdias attribute did not list the unknown in the first columm of the feature
train_data['RELAT_AB'] = train_data['RELAT_AB'].replace([9], np.nan)

#check columns with nan values again
null_columns_1 = (np.count_nonzero(train_data[train_data.columns].isnull(), axis = 0))
null_columns_df_1 = pd.DataFrame(null_columns_1,index=train_data.columns)
null_columns_df_1 = null_columns_df_1.sort_values(0,ascending=False)
null_columns_df_1 = null_columns_df_1.rename(columns={0:'no of missing values'})

#plotting bar graph for top 5 features with missing values
Features1 = null_columns_df_1.index[0:5].to_list()[::-1]
Quantity1 = null_columns_df_1['no of missing values'][0:5].to_list()[::-1]

plt.barh(Features1,Quantity1)
plt.title('FEATURES WITH MISSING VALUES')
plt.ylabel('FEATURES')
plt.xlabel('NO OF MISSING VALUES')
plt.show()


#create a dictionary with features as key and number of missing values as value
null_info_1 ={}
for i, j in zip(null_columns_1, column_names):
    if i > 0:
        null_info_1[j] = i
null_info_sorted_1 = dict(sorted(null_info_1.items(), key=lambda item: item[1], reverse=True))

#drop columns with 476524 or more missing values
columns_to_drop_1 = [i for i in null_info_1 if null_info_1[i] >= 476524]
train_data.drop(columns = columns_to_drop_1, inplace=True)

#determine the features that are in AZDIAS dataset but not in DIAS attribute
train_data_features_not_in_attributes = [i for i in train_data.columns.to_list() if i not in dias_att['Attribute'].to_list()]
len(train_data_features_not_in_attributes)

In [ ]:
#FURTHER DATA CLEANING

#dropping 'LP_STATUS_GROB', 'LP_FAMILIE_GROB' because similar to 'LP_STATUS_FEIN' and 'LP_FAMILIE_FEIN' 
train_data.drop(columns = ['LP_STATUS_GROB', 'LP_FAMILIE_GROB'], inplace=True)
#dropping LNR because it is a list of identification numbers
train_data.drop(columns = ['LNR'], inplace=True)

#reengineering features
#PLZ8_BAUMAX, how many family homes
train_data['PLZ8_BAUMAX'] = train_data['PLZ8_BAUMAX'].replace([5], 0)
#if person lives in business building or not
train_data['PLZ8_BAUMAX_BIZ'] = train_data['PLZ8_BAUMAX'].replace([0,1,2,3,4], [1,0,0,0,0])
#mainstream or avant-garde movement
train_data['PRAEGENDE_MAIN_OR_AVANT'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [0,1,0,1,0,1,1,0,1,0,1,0,1,0,1])
#replace 40ies,50ies,60ies,70ies,80ies,90ies movements with ordinal numbers
train_data['PRAEGENDE_JUGENDJAHRE'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [1,1,2,2,3,3,3,4,4,5,5,5,5,6,6])

#CAMEO_INTL_2015 and CAMEO_DEUG_2015 have string values ('X' and 'XX') that need to be replaced with NaN.
train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].replace(['X', 'XX'], np.nan)
train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].astype(float)
train_data['CAMEO_DEUG_2015'] = train_data['CAMEO_DEUG_2015'].replace(['X', 'XX'], np.nan)
train_data['CAMEO_DEUG_2015']  = train_data['CAMEO_DEUG_2015'].astype(float)

#creating a new feature indicating household wealth status from 'CAMEO HOUSEHOLD'
train_data['CAMEO_HOUSEHOLD'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                              41, 42, 43, 44, 45, 51, 52, 53, 54, 55],[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5])
#creating a new feature indicating family type from 'CAMEO HOUSEHOLD' 
train_data['CAMEO_FAMILY_TYPE'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,

#dropping CAMEO_INTL_2015                                                                          
train_data.drop(columns = ['CAMEO_INTL_2015'], inplace=True)                                                                             41, 42, 43, 44, 45, 51, 52, 53, 54, 55], [1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5])
#OST_WEST_KZ encode 'W' to 0 and '0' to 1
train_data['OST_WEST_KZ'] = train_data['OST_WEST_KZ'].replace(['W', 'O'], [0,1]) 
#one hot encoding of 'ANREDE_KZ' into 0 for male and 1 for female
train_data = pd.get_dummies(train_data, columns=['ANREDE_KZ'], drop_first=True)

In [ ]:
#for WOHNLAGE, replace 0 with nan
train_data['WOHNLAGE'] = train_data['WOHNLAGE'].replace([0], np.nan)
#create new WOHNLAGE feature, urban or rural region
train_data['WOHNLAGE_REGION'] = train_data['WOHNLAGE'].replace([1,2,3,4,5,7,8],[0,0,0,0,0,1,1])

#dropping CAMEO_DEU, because they are nominal features and have too many categories (44 categoris) 
train_data.drop(columns = ['CAMEO_DEU_2015'], inplace=True)
#dropping LP_LEBENSPHASE_FEIN, because they are nominal features and have too many categories (40 categoris)
train_data.drop(columns = ['LP_LEBENSPHASE_FEIN'], inplace=True)
#dropping D19_LETZTER_KAUF_BRANCHE, since it is duplicated information
train_data.drop(columns=['D19_LETZTER_KAUF_BRANCHE'], inplace=True)


#converting 'EINGEFUEGT_AM' to total months (1/1/2019 is used as the end date)
eingefuegt = np.empty(shape=(train_data.shape[0],1))
k = 0
train_data['EINGEFUEGT_AM'] = train_data['EINGEFUEGT_AM'].fillna(-1)
for i in train_data['EINGEFUEGT_AM']:
    if i != -1:

        yrmthday, zeros = i.split()
        mth_1 = (2019 - (int(yrmthday[0:4]) + 1)) * 12
        mth_2 = 12 - int(yrmthday[5:6])
        mth_3 = (30 - int(yrmthday[8:10])) / 30
        mth_total = mth_1 + mth_2 + mth_3
        eingefuegt[k] = mth_total
    else:
        eingefuegt[k] = -1
    k += 1
    
        
#remove 'EINGEFUEGT_AM' and create a new feature 'EINGEFUEGT_MODIFIED'
train_data.drop(columns =['EINGEFUEGT_AM'] , inplace=True)
train_data['EINGEFUEGT_MODIFIED'] = pd.DataFrame(eingefuegt)
#since i converted the NaN values to -1 earlier, i will need to replace the -1 value with mode value
train_data['EINGEFUEGT_MODIFIED'] = train_data['EINGEFUEGT_MODIFIED'].replace([-1],train_data['EINGEFUEGT_MODIFIED'].mode().values)

In [ ]:
#this function fills NaN values with mean value for continuous features and mode value for nominal/ordinal features
def impute_nan_values(df):  
    continuous_features = ['EINGEZOGENAM_HH_JAHR' , 'MIN_GEBAEUDEJAHR', 'KBA13_ANZAHL_PKW', 'ANZ_HAUSHALTE_AKTIV','VERDICHTUNGSRAUM', 'ANZ_STATISTISCHE_HAUSHALTE']
    
    #fill continuous features nan with mean value
    df[continuous_features] = df[continuous_features].fillna(np.mean(df[continuous_features]))

    #list of categorical/ordinal features
    all_features_except_continuous = df.columns.to_list()
    for i in continuous_features:
        all_features_except_continuous.remove(i)

    #fill categorical features nan with most frequent values
    for column in df[all_features_except_continuous].columns:
        df[column].fillna(df[column].mode()[0], inplace=True)
    
    return df

#fill AZDIAS and CUSTOMER NaN values with mode/mean values
train_data = impute_nan_values(train_data)